In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme()

import functions as myfunc

In [ ]:
nodes = np.loadtxt("Lösung/Netz1D_p.dat")
number_of_nodes = len(nodes)
element_indices = np.loadtxt("Lösung/Elemente1D_t.dat")

coefficients_matrix = np.loadtxt("Lösung/Netz1D_Matrix_K.dat")
rhs_matrix = np.loadtxt("Lösung/Netz1D_Vector_D.dat").reshape((number_of_nodes, 1))

def alpha(x):
    """
    Piecewise function for alpha(x):
    - 3 in [1.5, 2.7]
    - x^2 elsewhere
    """
    x = np.asarray(x)
    return np.where((1.5 <= x) & (x <= 2.7), 3.0, np.square(x))

def beta(x):
    """
    Piecewise function for beta(x):
    - x / (x+1) in [1, 2]
    - x^2 elsewhere
    """
    x = np.asarray(x)
    return np.where((1 <= x) & (x <= 2), x / (x + 1), np.square(x))

def rhs(x):
    """
    Right-hand side f(x):
    - x in [2, 4]
    - x + 1 elsewhere
    """
    x = np.asarray(x)
    return np.where((2 <= x) & (x <= 4), x, x + 1)

def gamma(x):
    """
    Robin issue function gamma(x)"""
    return x

def rho(x):
    """
    Robin issue function rhs rho(x)
    """
    x = np.asarray(x)
    return np.power(x, 3)

In [ ]:
DOMAIN_BOUNDARY = {
    "Lower Bound": 1,
    "Upper Bound": 4
}
BOUNDARY_CONDITION = {
    "Lower Bound": {
        "x": DOMAIN_BOUNDARY["Lower Bound"],
        "Gamma": gamma(DOMAIN_BOUNDARY["Lower Bound"]),
        "Rho": rho(DOMAIN_BOUNDARY["Lower Bound"])
        # "Phi": np.exp(DOMAIN_BOUNDARY["Lower Bound"])
    },
    "Upper Bound": {
        "x": DOMAIN_BOUNDARY["Upper Bound"],
        "Gamma": gamma(DOMAIN_BOUNDARY["Upper Bound"]),
        "Rho": rho(DOMAIN_BOUNDARY["Upper Bound"])
        # "Phi": np.exp(DOMAIN_BOUNDARY["Upper Bound"])
    }
}

BOUNDARY_CONDITION["Lower Bound"]["x Index"] = np.where(np.isclose(nodes, BOUNDARY_CONDITION["Lower Bound"]["x"]))[0][0]
BOUNDARY_CONDITION["Upper Bound"]["x Index"] =np.where(np.isclose(nodes, BOUNDARY_CONDITION["Upper Bound"]["x"]))[0][0]